In [1]:
import numpy as np
import yfinance as yf
import talib as ta

from decimal import Decimal
from ibapi.client import *
from ibapi.wrapper import *

In [ ]:
# orderId = 1
symbol = "VTI"
df = yf.download(symbol, start="2022-01-01")

# Create 200-period moving average
df['MA_10'] = ta.SMA(df['Close'], 10)

# Create 10-period moving average
df['MA_200'] = ta.SMA(df['Close'], 200)

# Create RSI_2 Column
df['RSI_2'] = ta.RSI(df['Close'], 2)

# Set long threshold
long_threshold = 10

# Enter long position
df['Long_Positions'] = np.where((df.RSI_2 < long_threshold) & (df.Close > df.MA_200), 1, np.nan)

# Close long position
df['Long_Positions'] = np.where(df.Close > df.MA_10, -1, df['Long_Positions'])

# Fill missing values
df = df.fillna(method="ffill")

In [ ]:
class TestApp(EClient, EWrapper):
    global df
    global symbol
    positions = {}
    
    def __init__(self):
        EClient.__init__(self, self)
        
 
    def nextValidId(self, orderId: OrderId):
        mycontract = Contract()
        mycontract.symbol = symbol
        mycontract.secType = "STK"   
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        
        self.reqContractDetails(orderId, mycontract)
        self.start()
 
    def position(self, account: str, contract: Contract, position: Decimal,
                      avgCost: float):
        super().position(account, contract, position, avgCost)
        self.positions[contract.symbol] = position
        
    def positionEnd(self):
        super().positionEnd()
        print("PositionEnd")
    
        
    def start(self):
        self.reqPositions()
        
    def stop(self):
        self.cancelPositions()
        self.done = True
        self.disconnect()


    def contractDetails(self, reqId: int, contractDetails: ContractDetails):
        global df
        global symbol
        
        print(contractDetails.contract)
        
        myorder = Order()
        myorder.orderId = reqId
        myorder.tif = "OPG"
        myorder.orderType = "MKT"
        myorder.totalQuantity = 10
        myorder.eTradeOnly = ''
        myorder.firmQuoteOnly = ''
        
         
        if df.tail(1)['Long_Positions'].values[0] == 1:
            myorder.action = "BUY"
            
        elif df.tail(1)['Long_Positions'].values[0] == -1 and self.positions[symbol] > 0:
            myorder.action = "SELL"
        else:
            pass
        
        
        self.placeOrder(myorder.orderId, contractDetails.contract, myorder)
        self.stop()
 
 
app = TestApp()
app.connect("127.0.0.1", 7497, 100)
app.run()